In [1]:
import json
import random

import nltk
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

#nltk.download('punkt_tab')

In [ ]:
"""# load the dataset
with open('data/healthcare.json','r',encoding='utf-8') as f:
    data = json.load(f)

intents = {'intents':[]}
#
for convo in data:
    tag = convo.get("agent_selected_tool", "general").replace(" ","_").lower()


    pattern = convo.get("user_1", "")
    response = convo.get("agent_initial_response", "")


    if pattern and response:
        intents['intents'].append({
            "tag":tag,
            "patterns":[pattern],
            "responses":[response]
        })

# Save the new dataset
with open("data/healthcare_intents.json", "w", encoding="utf-8") as f:
    json.dump(intents, f, indent=4)


print('Dataset intentions create with sucess!')"""
    








In [2]:
#
class ChatboModel(nn.Module):
    
    def __init__(self, input_size, output_size):
        super().__init__()

        self.fc1 = nn.Linear(input_size,512)
        self.fc2 = nn.Linear(512,256)
        self.fc3 = nn.Linear(256,128)
        self.fc4 = nn.Linear(128,output_size)
        
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self,x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x

        


In [ ]:
#
class ChabotAssistant:
    def __init__(self,intents_path, function_mappings = None):
        self.model = None
        self.intents_path = intents_path

        self.documents = []
        self.vocabulary = []
        self.intents = []
        self.intents_responses = {}

        self.function_mappings = function_mappings

        self.X = None
        self.y = None

        #
        for resource in ['wordnet', 'omw-1.4', 'punkt']:
            try:
                nltk.data.find(f'corpora/{resource}')
            except LookupError:
                 nltk.download(resource)

    #
    @staticmethod
    def tokenize_and_lemmatize(text):
        lemmatizer = nltk.WordNetLemmatizer()

        words = nltk.word_tokenize(text)
        words = [lemmatizer.lemmatize(word.lower()) for word in words]

        return words
    
    def bag_of_words(self, words):
        return[1 if word in words else 0 for word in self.vocabulary]
    
    #
    def parse_intents(self):
        #lemmatizer = nltk.WordNetLemmatizer()

        if os.path.exists(self.intents_path):
            with open(self.intents_path,'r', encoding= "utf-8") as f:
                intents_data = json.load(f)

            for intent in intents_data['intents']:
                if intent['tag'] not in self.intents:
                    self.intents.append(intent['tag'])
                    self.intents_responses[intent['tag']] = intent['responses']
                
                for pattern in intent['patterns']:
                    pattern_words = self.tokenize_and_lemmatize(pattern)
                    self.vocabulary.extend(pattern_words)
                    self.documents.append((pattern_words, intent['tag']))
                
            self.vocabulary = sorted(set(self.vocabulary))
        else:
            raise FileNotFoundError(f"???{self.intents_path}???")

    #
    def prepare_data(self):
        bags = []
        indices = []

        for words, intent in self.documents:
            bag = self.bag_of_words(words)
            intent_index = self.intents.index(intent)
            bags.append(bag)
            indices.append(intent_index)

            

        self.X = np.array(bags)
        self.y = np.array(indices)

        print(f"Prepared {len(self.X)} training examples.")

    #
    def train_model(self, batch_size, lr, epochs):
        X_tensor = torch.tensor(self.X,dtype = torch.float32)
        y_tensor = torch.tensor(self.y,dtype = torch.long)
        
        dataset = TensorDataset(X_tensor,y_tensor)
        loader  = DataLoader(dataset, batch_size=batch_size, shuffle=True)

        self.model = ChatboModel(self.X.shape[1],len(self.intents))

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.model.parameters(), lr = lr)

        loss_values = []

        print("\n Starting training...\n")

        for epoch in range(epochs):
            running_loss = 0.0

            for batch_X, batch_y in tqdm(loader,desc= f"Epoch {epoch+1}/{epochs}",leave=False):
                optimizer.zero_grad()
                outputs = self.model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()

            epoch_loss = running_loss / len(loader)
            loss_values.append(epoch_loss)
            print(f"Epoch {epoch+1}/{epochs} | Loss: {epoch_loss:.4f}")

        plt.figure(figsize=(8,5))
        plt.plot(loss_values, label= "Training Loss", color = "blue")
        plt.title("Training Loss Curve")
        plt.xlabel("Epochs")
        plt.ylabel("Loss")
        plt.legend()
        plt.grid(True)
        plt.show

    #
    def save_model(self,model_path, dimensions_path):
        torch.save(self.model.state_dict(), model_path)

        with open(dimensions_path,"w") as f:
            json.dump({'input_size': self.X.shape[1],'output_size':len(self.intents)},f, indent=4)

    def load_model(self,model_path, dimensions_path):
        with open(dimensions_path,"r") as f:
            dimensions = json.load(f)

        self.model = ChatboModel(dimensions['input_size'], dimensions['output_size'])
        self.model.load_state_dict(torch.load(model_path))

    #
    def process_message(self, input_message):
        words = self.tokenize_and_lemmatize(input_message)
        bag = self.bag_of_words(words)
        bag_tensor = torch.tensor([bag], dtype=torch.float32)

        self.model.eval()
        with torch.no_grad():
            predictions = self.model(bag_tensor)

        predicted_class_index = torch.argmax(predictions, dim=1).item()
        predicted_intent = self.intents[predicted_class_index]

        if self.function_mappings and predicted_intent in self.function_mappings:
            self.function_mappings[predicted_intent]()

        if self.intents_responses[predicted_intent]:
            return random.choice(self.intents_responses[predicted_intent])
        else:
            return "?????."






    

In [25]:
#
if __name__== '__main__':
    assistant = ChabotAssistant("data/healthcare_intents.json")
    assistant.parse_intents()
    assistant.prepare_data()
    #assistant.train_model(batch_size=32, lr =0.001, epochs=150)    

    #assistant.save_model('chat_model.pth','dimensions.json')


    assistant.load_model('chat_model.pth','dimensions.json')

    while True:
        message = input("You: ")
        if message.lower() == "/quit":
            print("Chatbot: Goodbye! ")
            break
        print("Chatbot:", assistant.process_message(message))


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Calixte\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Calixte\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Calixte\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Prepared 683 training examples.
Chatbot: I can look into that for you. Can you provide your name and the claim number?
Chatbot: Certainly. Which department are you trying to reach?
Chatbot: Our clinic operates Monday through Saturday.
Chatbot: It’s best to book an appointment so a doctor can evaluate your symptoms.
Chatbot: I’m not a doctor, but I can help you understand possible causes. How long have you been feeling this way?
Chatbot: Sure! Can you please provide your name and preferred date and time?
Chatbot: Goodbye! 
